In [1]:
import requests
from time import time
import pickle

# 1. Clustering covid topics

In [2]:
topicMap = dict()
topicMap['generic'] = ["corona", "#corona", "coronavirus", "#coronavirus", "covid", "#covid", "covid19", "#covid19", "covid-19", "#covid-19", "sarscov2", "#sarscov2", "sars cov2", "sars cov 2", "covid_19", "#covid_19", "pandemic", "#pandemic" ,"#coronaupdate"]
topicMap['covid_count'] = ["#ncov", "ncov", "#ncov2019", "ncov2019", "2019-ncov", "#2019-ncov", "#2019ncov", "2019ncov"]
topicMap['government_norms'] =  ["#lockdown", "lockdown"]
topicMap['safety_guidelines'] =  ["social distancing", "#socialdistancing", "wash ur hands", "wash your hands", "#washurhands", "#washyourhands" , "wearamask", "#wearamask", "wear a mask"]
topicMap['public_tackling_covid'] = ["quarantine", "#quarantine", "#selfisolating", "self isolating" , "flatten the curve", "flattening the curve", "#flatteningthecurve", "#flattenthecurve"]
topicMap['vaccines_and_medicines'] = ["vaccine", "vaccines", "#vaccine", "#vaccines", "corona vaccine", "corona vaccines", "#coronavaccine", "#coronavaccines"]
topicMap['herd_immunity'] = ["herd immunity", "#herdimmunity"]
topicMap['working_from_home'] = ["#stayathome", "#stayhome","#wfh", "work from home", "#workfromhome", "working from home", "#workingfromhome", "#stayhomestaysafe", "#hometasking"]
topicMap['protective_equipment'] = ["n95" , "ppe", "#ppe", "#n95", "hand sanitizer", "#handsanitizer" ,"#faceshields", "#masks4all", "face shield", "#faceshield", "face shields"] 
topicMap['people_not_following_measures'] = ["#covidiots", "covidiots"]
topicMap['comparison_with_flu']  = ["pneumonia", "#pneumonia"]
topicMap['covid_nick_names'] = ["chinese virus", "#chinesevirus", "wuhan virus", "#wuhanvirus", "kung flu", "#kungflu"]
topicMap['health_workers'] = ["health worker", "#healthworker", "health workers", "#healthworkers", "#frontlineheroes", "#coronawarriors"]
topicMap['home_schooling'] = ["#homeschool", "#homeschooling"]
                                    
pickle.dump(topicMap, open( "topics.pkl", "wb" ) )

# 2. Fetching Topic by Topic and in batches of 1000 and dumping them as jsons

In [49]:
def build_data_dict(topic):
    data_dict = dict()
    query ='text:' + ','.join(topic)
    data_dict['q'] = query
    data_dict['rows'] = 1000
    return data_dict

In [50]:
import os
import json

start = time()

batch = 0

url = 'http://awesome-hw.sdsc.edu:8983/solr/political_data/select'

for topic in topicMap.keys():
    for i in range(0,150000,1000):
        data_dict = build_data_dict(topicMap[topic])
        data_dict['wt'] = 'json'
        data_dict['start'] = i
        r = requests.post(url,data = data_dict)
        if not os.path.exists('political_tweets/' + topic):
            os.mkdir('political_tweets/' + topic)
        if eval(r.content)['response']['docs']:
            with open('political_tweets/' + topic +'/tweets_'+str(batch)+ '.json','wb') as f:
                f.write(r.content)
        batch += 1 
        #print("Completed batch Number %d"%batch)

end = time()

print('Time taken for entire batch rows: %f'%(end-start))

Time taken for entire batch rows: 2252.889197


In [15]:
import os
os.listdir('./political_tweets/')[:15]

['public_tackling_covid',
 'covid_count',
 'government_norms',
 'safety_guidelines',
 'protective_equipment',
 'vaccines_and_medicines',
 'herd_immunity',
 '.Rhistory',
 'home_schooling',
 'working_from_home',
 'health_workers',
 'comparison_with_flu',
 'covid_nick_names',
 'people_not_following_measures',
 'generic']

In [14]:
os.listdir('./political_tweets/vaccines_and_medicines/')[:5]

['tweets_751.json',
 'tweets_758.json',
 'tweets_776.json',
 'tweets_767.json',
 'tweets_884.json']

### For each topic create create labels and add emoji attribute

In [3]:
import pandas as pd
from glob import glob
import ipywidgets as widgets

#### Twitter setup

In [4]:
#!pip install tweepy
import os
import tweepy as tw
import text2emotion as te

##  Keys
consumer_key= '#'
consumer_secret= '#'
access_token= '#'
access_token_secret= '#'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

[nltk_data] Downloading package stopwords to /home/akash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/akash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/akash/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
all_topics = os.listdir('political_tweets/')

In [14]:
all_topics.remove('.Rhistory')
all_topics.remove('pickles')
all_topics.remove('people_not_following_measures')
all_topics.remove('covid_count')
all_topics.remove('government_norms')
all_topics.remove('public_tackling_covid')
all_topics.remove('safety_guidelines')

In [18]:
all_topics[0] , all_topics[3] = all_topics[3] , all_topics[0]

In [23]:
import json

In [27]:
def fxget_emotionscore(x):
    return te.get_emotion(x)

In [28]:
def parse_tweet_object(twobject, twdf, usrdf, pstdf, tgdf, mntndf, rtwtsdf, qtsdf, cntnsdf, rplsdf,topic):
    tid=twobject["id"]
    
    if "extended_tweet" in twobject.keys():
        twtext=twobject["extended_tweet"]["full_text"]
        thashtags=twobject["extended_tweet"]["entities"]["hashtags"]
        umentions=twobject["extended_tweet"]["entities"]["user_mentions"]
    else:
        twtext=twobject["text"]
        thashtags=twobject["entities"]["hashtags"]
        umentions=twobject["entities"]["user_mentions"]
    
    emotion_dict = fxget_emotionscore(twtext)        
    twdf.loc[tid,"happy"]= emotion_dict['Happy']
    twdf.loc[tid,"angry"]=emotion_dict['Angry']
    twdf.loc[tid,"surprise"]=emotion_dict['Surprise']
    twdf.loc[tid,"sad"]=emotion_dict['Sad']
    twdf.loc[tid,"fear"]=emotion_dict['Fear']
    twdf.loc[tid,"topic"]=topic
    
    if tid not in twdf.keys():
        twdf.loc[tid,"text"]=twtext
        twdf.loc[tid,"date"]=pd.to_datetime(twobject["created_at"])
        for key in replykeys:
            twdf.loc[tid,key] = twobject[key]
            
        #set these fields to false by default
        twdf.loc[tid,"retweet"] = False
        twdf.loc[tid,"quote"] = False
        twdf.loc[tid,"reply"] = False
    else:
        for key in replykeys:
            twdf.loc[tid,key] = max(twdf.loc[key], twobject[key])
   
    uid=twobject["user"]["id"]

    if uid not in usrdf.index:
        usrdf.loc[uid,"screen_name"]=twobject["user"]["screen_name"]
        usrdf.loc[uid,"created"]=pd.to_datetime(twobject["user"]["created_at"])
        usrdf.loc[uid,"description"]=twobject["user"]["description"]
        usrdf.loc[uid,"location"]=twobject["user"]["location"]

    #keep track of maximum number of friends found through database
    ufriends=twobject["user"]["friends_count"]
    
    if uid in usrdf.index:
        usrdf.loc[uid,"friends_count"] = \
            max(ufriends,usrdf.loc[uid,"friends_count"])
    else:
        usrdf.loc[uid,"friends_count"] = ufriends

    np=len(pstdf)
    pstdf.loc[np,"uID"] = uid
    pstdf.loc[np,"tID"] = tid

    #extract hashtags
    nh=len(tgdf)
    for p,ht in enumerate(thashtags):
        #twitter hashtags are not case sensitive, so they are converted to lowercase
        tgdf.loc[nh+p,"hashtag"] = ht["text"].lower()
        tgdf.loc[nh+p,"tID"] = tid

    #extract user mentions
    nu=len(mentionsdf)

    for p,ment in enumerate(umentions):
        mntndf.loc[nu+p,"tID"] = tid
        mntndf.loc[nu+p,"uID"] = ment["id"]
        
    #extract retweets
    if "retweeted_status" in twobject.keys():
        nr=len(rtwtsdf)
        rtwtsdf.loc[nr,"tID"] = tid
        tid_re=twobject["retweeted_status"]["id"]
        rtwtsdf.loc[nr,"tID_re"] = tid_re
        rtwtsdf.loc[nr,"topic"]=topic
        twdf.loc[tid,"retweet"] = True
    
    if "quoted_status" in twobject.keys():
        nq=len(qtsdf)
        qtsdf.loc[nq,"tID"] = tid
        qtsdf.loc[nq,"tID_qo"] = twobject["quoted_status"]["id"]
        qtsdf.loc[nq,"topic"]=topic
        twdf.loc[tid,"quote"] = True
 
    urls=twobject["entities"]["urls"]
    for url in urls:
        if "twitter.com/i/web/status/" not in url["expanded_url"]:
            nc=len(cntnsdf)
            cntnsdf.loc[nc,"url"] = url["url"]
            cntnsdf.loc[nc,"tid"] = tid
            cntnsdf.loc[nc,"expanded_url"] = url["expanded_url"]
            
        #else:
        #    tid_ln=int(url["expanded_url"].split("twitter.com/i/web/status/")[1])
        #    nl=len(linkedtweetsdf)
        #    linkedtweetsdf.loc[nl,"tid"] = tid
        #    linkedtweetsdf.loc[nl,"tid_ln"] = tid_ln
    
    #if "in_reply_to_status_id" in twobject.keys():
    if type(twobject["in_reply_to_status_id"]) != type(None):

        nl=len(rplsdf)
        rplsdf.loc[nl,"tID"] = tid
        rplsdf.loc[nl,"tID_rp"] = twobject["in_reply_to_status_id"]
        rplsdf.loc[nl,"uID_rp"] = twobject["in_reply_to_user_id"]
        rplsdf.loc[nl,"topic"]=topic
        twdf.loc[tid, "reply"] = True

In [29]:
def parse_all_tweets(jsonfile,tweetdf,userdf,postdf,mentionsdf, retweetsdf,quotesdf,tagsdf,containsdf,repliesdf,topic):
    tweet=json.load(open(jsonfile, "rb"))
    alltweets=tweet["response"]["docs"]
    ntweet=len(alltweets)
   
    for n in range(ntweet):
        atweet=alltweets[n]
        fulltweet=eval(atweet["fulltweets"])
        
        parse_tweet_object(fulltweet,
                           tweetdf, userdf, postdf, tagsdf, mentionsdf, 
                           retweetsdf, quotesdf, containsdf, repliesdf,topic)
        
        if "quoted_status" in fulltweet.keys():
            parse_tweet_object(fulltweet["quoted_status"],
                               tweetdf, userdf, postdf, tagsdf, mentionsdf, 
                               retweetsdf, quotesdf, containsdf, repliesdf,topic)
            
        if "retweet_status" in fulltweet.keys():
            parse_tweet_object(fulltweet["retweet_status"],
                               tweetdf, userdf, postdf, tagsdf, mentionsdf, 
                               retweetsdf, quotesdf, containsdf, repliesdf,topic)

In [ ]:
replykeys=["reply_count", "retweet_count", "quote_count", "favorite_count"]

for i in range(0,len(all_topics)):
    tweetfiles=glob("political_tweets/" + all_topics[i] + "/tweets_*.json")
    tweetdf=pd.DataFrame(columns=["text", "date","happy","angry","surprise","sad","fear","topic"] + replykeys + ["retweet", "quote", "reply"])
    userdf=pd.DataFrame(columns=["screen_name", "created","description",
                            "location", "friends_count"])
    #these may be reduntant as the hashtag and url serves as primary key
    #hashtags=pd.DataFrame(columns=["hashtag"])
    #url=pd.DataFrame(columns="url")
    
    #these relations will be edges in the graph
    postdf=pd.DataFrame(columns=["uID","tID","topic"])
    
    # tweets mentioning users
    mentionsdf=pd.DataFrame(columns=["tID","uID"])
    retweetsdf=pd.DataFrame(columns=["tID","tID_re","topic"])
    quotesdf=pd.DataFrame(columns=["tID", "tID_qo","topic"])
    tagsdf=pd.DataFrame(columns=["hashtag","tID"])
    containsdf=pd.DataFrame(columns=["url","tID", "expanded_url"])
    repliesdf=pd.DataFrame(columns=["tID","tID_rp", "uID_rp","topic"])
    
    #linkedtweetsdf=pd.DataFrame(columns=["tID","tID_ln"])
    #for file in tweetfiles:
    
    #parse_tweet_info(file)
    from ipywidgets import IntProgress
    from IPython.display import display

    wb = IntProgress(min=0, max=len(tweetfiles)) 
    display(wb)

    for n,file in enumerate(tweetfiles):
        pcent = float(n+1)/len(tweetfiles) * 100.
        print("\rReading %s ...  (%4.2f%%) " % (file, pcent), end="")
        parse_all_tweets(file,tweetdf,userdf,postdf,mentionsdf, retweetsdf,quotesdf,tagsdf,containsdf,repliesdf,all_topics[i])
        wb.value = n+1
        
    alldf={"tweets": tweetdf, "users": userdf, "posts": postdf, "mentions": mentionsdf, "retweets": retweetsdf,
      "quotes": quotesdf, "tags": tagsdf, "contains": containsdf,"replies":repliesdf}
    
    if not os.path.exists('political_tweets/pickles'):
        os.mkdir('political_tweets/pickles')
    pickle.dump(alldf, open("political_tweets/pickles/" + all_topics[i] + ".pkl", "wb"))
    
    print(" done.")

IntProgress(value=0, max=3)

Reading political_tweets/home_schooling/tweets_1950.json ...  (100.00%)  done.


IntProgress(value=0, max=150)

Reading political_tweets/vaccines_and_medicines/tweets_805.json ...  (53.33%) 

In [17]:
import pickle
all_df = pickle.load(open('political_tweets/pickles/vaccines_and_medicines.pkl','rb'))

In [20]:
all_df['tweets'].head()

,text,date,happy,angry,surprise,sad,fear,topic,reply_count,retweet_count,quote_count,favorite_count,retweet,quote,reply
1315385995676839938,RT @jkfecke: That is not how vaccines work. ht...,2020-10-11 20:17:14+00:00,0,0,0,0,1,vaccines_and_medicines,0,0,0,0,True,True,False
1315327582620053504,"On ABC News this morning, Eric Trump said his ...",2020-10-11 16:25:07+00:00,0,0,1,0,0,vaccines_and_medicines,83,191,51,739,False,False,False
1313389136724725760,@realDonaldTrump Duck the vaccines if the trea...,2020-10-06 08:02:26+00:00,0,0,0,1,0,vaccines_and_medicines,0,0,0,0,False,False,True
1315856227528667136,"RT @DarlaShine: ""You tell everybody my dad wil...",2020-10-13 03:25:46+00:00,0,0,0,1,0,vaccines_and_medicines,0,0,0,0,True,True,False
1315439303489736704,Shannon Kroner got the opportunity to meet wit...,2020-10-11 23:49:04+00:00,0.33,0,0.67,0,0,vaccines_and_medicines,27,189,33,401,False,False,False


In [21]:
all_df['retweets'].head()

,tID,tID_re,topic
0,1315385995676839938,1315385063362699270,vaccines_and_medicines
1,1315856227528667136,1315772869788479488,vaccines_and_medicines
2,1315185398000254976,1315170291052679168,vaccines_and_medicines
3,1315373970632441863,1315170291052679168,vaccines_and_medicines
4,1313196980303212551,1313192155817685000,vaccines_and_medicines


In [26]:
all_df['posts'][['uID','tID']].head()

,uID,tID
0,17138386,1315385995676839938
1,15464697,1315327582620053504
2,1212935332192112640,1313389136724725760
3,1250465080904265734,1315856227528667136
4,1499607608,1315439303489736704


In [29]:
all_df['replies'].head()

,tID,tID_rp,uID_rp,topic
0,1313389136724725760,1313267615083761665,25073877,vaccines_and_medicines
1,1313394196376702976,1313267143232942081,25073877,vaccines_and_medicines
2,1313395528026083328,1313375957319741441,984974481683722241,vaccines_and_medicines
3,1313924088079892480,1313843369785856002,25073877,vaccines_and_medicines
4,1313296452551086081,1313267615083761665,25073877,vaccines_and_medicines
